In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

### Connecting to MongoDB

**Creating Spark connector**

In [5]:
spark = SparkSession.builder.appName("pipeline") \
    .config('spark.mongodb.input.uri', 'mongodb://localhost:27017/usa_training_tweets_28_06.usa_training_tweets_collection_23_06') \
    .config('spark.mongodb.output.uri', 'mongodb://localhost:27017/usa_training_tweets_28_06.usa_training_tweets_collection_23_06') \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1') \
    .config('spark.mongodb.input.partitioner', 'MongoPaginateBySizePartitioner') \
    .getOrCreate()

**Creating Spark dataframe on top of DB**

In [13]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

**Creating temp table for SQL queries**

In [17]:
# df.createOrReplaceTempView("temp")

In [18]:
# df.printSchema()

**Sample query**

In [15]:
sample = spark.sql('SELECT * FROM temp LIMIT 10')

In [19]:
# sample.collect()

In [11]:
import nltk
from nltk.corpus import stopwords
import re

In [20]:
StopWords = stopwords.words("english")

tokens = df.select('text').rdd \
    .map(lambda x: x[0]) \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \


In [21]:
tokens.collect()

[['tweaking', 'sayin', 'names', 'cars'],
 ['Thinking', 'bout', 'oovoo', 'video', 'call', 'messenger'],
 ['That', 'late', 'night', 'blunt', 'listening', 'Entropy'],
 ['Speak'],
 ['dumb', 'Urban', 'Dictionary', 'source', 'Chub'],
 ['deliver', 'kush'],
 ['photography',
  'keeps',
  'getting',
  'back',
  'need',
  'Bats',
  'flying',
  'around'],
 ['went',
  'worst',
  'place',
  'Houston',
  'sheesh',
  'members',
  'policy',
  'checked'],
 ['biting', 'cream', 'licks'],
 [],
 ['wanna', 'pretty', 'Where'],
 ['friends', 'girl', 'friends'],
 ['fuckin', 'pastor', 'daughter', 'jesus', 'starts', 'song', 'like', 'fuck'],
 ['need', 'carti'],
 ['Keep', 'Achieving', 'young', 'King'],
 ['definitely', 'year', 'baby', 'laughter', 'Dear'],
 ['hilarity', 'Dump', 'Funny', 'Picture', 'Dump', 'Pics'],
 ['Funny', 'creeps', 'faster', 'venom', 'rushing', 'threw', 'blood'],
 ['Faccs'],
 ['Maybe', 'doctor', 'able', 'help'],
 ['Nice'],
 ['fuck'],
 ['shit'],
 ['Congrats', 'scoring', 'first', 'career', 'Many'],
 